## RAG pipeline from Scratch

#### The Goal of RAG is to take information and pass it to an LLM so that it can generate output based on that information 

* Retrieval - Find relavent info from a query
* Augmented - Augment out input to the LLM
* Generation - Generative output from LLM

## STEPS

1. Open a PDF document (even a collection of PDFs)
2. Format the text of the PDF textbook ready for an embedding model.
3. Embed all of the chunks of text in the textbook and turn them into numerical embeddings which can store for later.
4. Build a retrieval system that uses vector search to find relevant chunck of text based ona aquery
5. Create a prompt that incorporates the relevant pieces of text.
6. Generat an answer to a query based on the passages of the textbook with an LLM

In [1]:
# downloading requirements
!pip install PyMuPDF
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 23.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 63.0 MB/s eta 0:00:00:00:0100:01


In [2]:
# Step 1
import os

# get pdf path
pdf_path = "/kaggle/input/nutrition-rag/human-nutrition-text.pdf"

In [3]:
# Step 2

import fitz
from tqdm.auto import tqdm 

def text_formatter(text: str) ->str:
    cleaned_text = text.replace("\n"," ").strip()
    
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    
    for page_number, page in tqdm(enumerate(doc)):
#         print(page_number)
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number - 41,
                               "page_char_count": len(text),
                               "page_word_count": len(text.split(" ")),
                               "page_sentence_count_raw": len(text.split(".")),
                               "page_token_count": len(text)/4, # 1 toke  is ~4 characters
                               "text":text})
        
    return pages_and_texts
    
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [4]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,3,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [5]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,14.180464,287.148593
std,348.86387,560.441673,95.747365,9.544587,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,8.000000,190.687500
50%,562.50000,1232.500000,215.000000,13.000000,308.125000
75%,864.25000,1605.250000,271.250000,19.000000,401.312500
max,1166.00000,2308.000000,429.000000,82.000000,577.000000


In [6]:
# Splitting text in page into sentences, using NLP lib - spaCy
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item['text']).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    #Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [7]:
import random

random.sample(pages_and_texts, k=1)

[{'page_number': 75,
  'page_char_count': 920,
  'page_word_count': 160,
  'page_sentence_count_raw': 9,
  'page_token_count': 230.0,
  'text': '“Segmentatio n” by  OpenStax  College / CC  BY 3.0  forward. Segmentation from circular muscle contraction slows  movement in the small intestine by forming temporary “sausage  link” type of segments that allows chyme to slosh food back and  forth in both directions to promote mixing of the chyme and  enhance absorption of nutrients (Figure 2.7 “Segmentation”). Almost  all the components of food are completely broken down to their  simplest units within the first 25 centimeters of the small intestine.  Instead of proteins, carbohydrates, and lipids, the chyme now  consists of amino acids, monosaccharides, and emulsified  components of triglycerides.  Figure 2.7 Segmentation  The third step of digestion (nutrient absorption) takes place mainly  in the remaining length of the small intestine, or ileum (> 5 meters).  The way the small intestine i

In [8]:
# STEP 3

# Chucking our sentences together
# Chucking - grouping sentences into groups

# We do this because, to make our text chunks fit into our embedding model context window

num_sentence_chunk_size = 10

# eg: [25] => [10,10,5] 

def split_list(input_list:list[str],
              slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list),slice_size)]

# Testing
test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [9]:
# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list = item["sentences"],
                                        slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [10]:
random.sample(pages_and_texts, k=1)

[{'page_number': 1020,
  'page_char_count': 1502,
  'page_word_count': 261,
  'page_sentence_count_raw': 16,
  'page_token_count': 375.5,
  'text': 'Why Irradiate Food?  Irradiation can serve many purposes.  • Prevention of Foodborne Illness – to effectively eliminate  organisms that cause foodborne illness, such as Salmonella and  E. coli.  • Preservation – to destroy or inactivate organisms that cause  spoilage and decomposition and extend the shelf life of foods.  • Control of Insects – to destroy insects in or on tropical fruits  imported into the United States. Irradiation also decreases the  need for other pest-control practices that may harm the fruit.  • Delay of Sprouting and Ripening – to inhibit sprouting (e.g.,  potatoes) and delay ripening of fruit to increase longevity.  • Sterilization – irradiation can be used to sterilize foods, which  can then be stored for years without refrigeration. Sterilized  foods are useful in hospitals for patients with severely impaired  immu

In [11]:
# Split each chunk into its own item
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [12]:
# View a random sample
random.sample(pages_and_chunks, k=1)

[{'page_number': 152,
  'sentence_chunk': 'Image by Allison Calabrese / CC BY 4.0 and negatively charged electrolytes are called anions. For example, in water sodium chloride (the chemical name for table salt) dissociates into sodium cations (Na+) and chloride anions (Cl−). Solutes refers to all dissolved substances in a fluid, which may be charged, such as sodium (Na+), or uncharged, such as glucose. In the human body, water and solutes are distributed into two compartments: inside cells, called intracellular, and outside cells, called extracellular. The extracellular water compartment is subdivided into the spaces between cells also known as interstitial, blood plasma, and other bodily fluids such as the cerebrospinal fluid which surrounds and protects the brain and spinal cord (Figure 3.2 “Distribution of Body Water”). The composition of solutes differs between the fluid compartments. For instance, more protein is inside cells than outside and more chloride anions exist outside of c

In [13]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,734.83,112.72,183.71
std,347.79,447.43,71.07,111.86
min,-41.00,12.00,3.00,3.00
25%,280.50,315.00,45.00,78.75
50%,586.00,746.00,114.00,186.50
75%,890.00,1118.50,173.00,279.62
max,1166.00,1831.00,297.00,457.75


In [14]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 25.5 | Text: http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=521   996 | The Major Types of Foodborne Illness
Chunk token count: 13.0 | Text: US Department of Agriculture, 1136 | Food Insecurity
Chunk token count: 30.0 | Text: 2011.  https:/ /www.ers.usda.gov/publications/pub- details/?pubid=44909. Accessed April 15, 2018. 1138 | Food Insecurity
Chunk token count: 29.25 | Text: You can view it online here: http:/ /pressbooks.oer.hawaii.edu/ humannutrition2/?p=55   28 | Lifestyles and Nutrition
Chunk token count: 16.0 | Text: PART II CHAPTER 2. THE HUMAN BODY Chapter 2. The Human Body | 53


In [15]:
# filter our DataFrame for rows with under 30 tokens

pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [16]:
# Embedding our text chunks

# using all-mpnet-base-v2 sentence transformer

!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00 0:00:01


In [17]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2',
                           device = "cuda")

2024-06-22 12:29:30.072123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 12:29:30.072234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 12:29:30.230624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
embed = embedding_model.encode(sentences)

embed

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[ 0.0225026 , -0.07829171, -0.02303073, ..., -0.00827931,
         0.02652686, -0.00201899],
       [ 0.04170235,  0.00109742, -0.0155342 , ..., -0.02181628,
        -0.0635936 , -0.00875286]], dtype=float32)

In [30]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

text_chunks[0]

'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE'

In [32]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks)
text_chunk_embeddings

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

RuntimeError: Invalid argument

In [20]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"

In [21]:
text_chunks_and_embeddings_df.shape

(1680, 5)

In [22]:
len(text_chunk_embeddings)

NameError: name 'text_chunk_embeddings' is not defined

In [ ]:
new_column_list_cpu = [tensor.cpu().numpy() for tensor in text_chunk_embeddings]
text_chunks_and_embeddings_df["embedding"] = new_column_list_cpu
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# Import saved file and view
text_chunks_and_embedding_df = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df.head()

#### NOTE: In this notebook, Haven't used Vector databases

*  In RAG, documents and queries are represented as high-dimensional vectors using embeddings
* Vector databases, like FAISS, Milvus, or Pinecone, are optimized for performing efficient similarity searches. They can quickly find the nearest vectors (similar documents) to a given query vector using algorithms like Approximate Nearest Neighbors (ANN).

#### Approximate Nearest Neighbors (ANN) Search:

* Indexing: Before searching, the database creates an index using ANN algorithms. This index organizes the vectors in a way that makes it efficient to search for nearest neighbors.
* Partitioning and Probing: The index partitions the vector space into multiple smaller regions or clusters. When a query is made, the search algorithm probes a subset of these regions that are most likely to contain similar vectors.
* Reduced Comparisons: Instead of comparing the query vector against all 10M vectors, the search algorithm only compares it against vectors in the selected regions. For example, it might compare the query vector with 1 million (1M) vectors that are in the most promising regions.

## Run from here

In [ ]:
!pip install -U sentence-transformers

In [33]:
# STEP 4
# RAG Search and Answer

# Comparing embedding is called similarity search or vector search

import torch
import numpy as np
import pandas as pd

text_chunks_and_embedding_df = pd.read_csv("/kaggle/input/nutrition-rag/text_chunks_and_embeddings_df.csv")
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242601e-02 9.02282074e-02 -5.09547768e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156083e-02 5.92139438e-02 -1.66167226e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 2.79801711e-02 3.39814313e-02 -2.06426643e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 6.82566911e-02 3.81274782e-02 -8.46855436e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264196e-02 -8.49765539e-03 9.57158115e-...


In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [36]:
# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1680, 768])

In [37]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
# Step to peforming search

# 1. Define the query
query = "macronutrients functions"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples 
query_embedding = embedding_model.encode(query, convert_to_tensor=True)
query_embedding

Query: macronutrients functions


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([ 3.6350e-02, -5.8203e-02, -1.8262e-02,  1.4228e-02,  1.3136e-02,
         4.3278e-02, -3.3172e-02, -1.1831e-02, -4.3201e-02, -6.8741e-02,
        -1.8463e-02, -1.5240e-02,  1.9626e-02,  4.7535e-02,  2.7343e-02,
        -7.1382e-03,  1.0865e-02, -2.5707e-02, -7.7592e-02, -5.8880e-03,
        -3.2791e-02,  2.0640e-02,  1.3843e-02, -6.2041e-03, -2.2706e-02,
         3.9274e-02,  4.0431e-02,  5.1309e-03,  1.1578e-02, -1.1135e-02,
        -1.9891e-02, -1.8691e-03, -4.9330e-02, -8.7934e-02,  1.5259e-06,
        -3.3809e-02, -4.7408e-02,  1.7066e-02, -7.5259e-02,  3.4821e-02,
         2.9377e-02, -3.2112e-02, -1.6509e-02,  7.3573e-03,  4.6359e-02,
        -4.1220e-03,  1.4522e-02,  3.7190e-04, -6.3193e-02,  1.9782e-02,
         2.9439e-02,  4.7811e-02,  1.6230e-02, -1.6853e-02,  2.3415e-02,
         4.1553e-02,  8.8754e-03,  2.3375e-02, -3.9122e-02,  3.5091e-02,
         6.1852e-02,  3.5322e-02, -1.1847e-03, -8.1968e-03,  4.3451e-02,
         5.5786e-02, -2.3311e-02, -3.3593e-02, -2.1

* For similarity -> not used cosine similarity, used dot product

Only difference in cosine sim and dot product is, cosine sim has normalization step, but we are already getting normalized embeddings from our embedding model

#### Dot Product
* Measure of magnitude and direction between two vectors
* Vectors that are aligned in direction and magnitude have a higher positive value
* Vectors that are opposite in direction and magnitude have a higher negative value

In [39]:
# 3. Get similarity scores with the dot product (we'll time this for fun)
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Time take to get scores on 1680 embeddings: 0.00385 seconds.


torch.return_types.topk(
values=tensor([0.6926, 0.6738, 0.6646, 0.6536, 0.6473], device='cuda:0'),
indices=tensor([42, 47, 41, 51, 46], device='cuda:0'))

In [40]:
# Define helper function to print wrapped text, so it doesn't print a whole text chunk as a single line 
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [41]:
print(f"Query: '{query}'\n")
print("Results:")
# Loop through zipped together scores and indicies from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    # Print the page number too so we can reference the textbook further (and check the results)
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

Query: 'macronutrients functions'

Results:
Score: 0.6926
Text:
Macronutrients Nutrients that are needed in large amounts are called
macronutrients. There are three classes of macronutrients: carbohydrates,
lipids, and proteins. These can be metabolically processed into cellular energy.
The energy from macronutrients comes from their chemical bonds. This chemical
energy is converted into cellular energy that is then utilized to perform work,
allowing our bodies to conduct their basic functions. A unit of measurement of
food energy is the calorie. On nutrition food labels the amount given for
“calories” is actually equivalent to each calorie multiplied by one thousand. A
kilocalorie (one thousand calories, denoted with a small “c”) is synonymous with
the “Calorie” (with a capital “C”) on nutrition food labels. Water is also a
macronutrient in the sense that you require a large amount of it, but unlike the
other macronutrients, it does not yield calories. Carbohydrates Carbohydrates
are 

#### Functions for the pipeline

In [43]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    # Get dot product scores on embeddings
    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    print("Results:")
    # Loop through zipped together scores and indicies
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        # Print the page number too so we can reference the textbook further and check the results
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")

In [44]:
query = "symptoms of pellagra"

# Get just the scores and indices of top related results
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get scores on 1680 embeddings: 0.00027 seconds.


(tensor([0.5000, 0.3741, 0.2959, 0.2793, 0.2721], device='cuda:0'),
 tensor([ 822,  853, 1536, 1555, 1531], device='cuda:0'))

In [45]:
# Print out the texts of the top scores
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Time taken to get scores on 1680 embeddings: 0.00017 seconds.
Query: symptoms of pellagra

Results:
Score: 0.5000
Niacin deficiency is commonly known as pellagra and the symptoms include
fatigue, decreased appetite, and indigestion.  These symptoms are then commonly
followed by the four D’s: diarrhea, dermatitis, dementia, and sometimes death.
Figure 9.12  Conversion of Tryptophan to Niacin Water-Soluble Vitamins | 565
Page number: 565


Score: 0.3741
car. Does it drive faster with a half-tank of gas or a full one?It does not
matter; the car drives just as fast as long as it has gas. Similarly, depletion
of B vitamins will cause problems in energy metabolism, but having more than is
required to run metabolism does not speed it up. Buyers of B-vitamin supplements
beware; B vitamins are not stored in the body and all excess will be flushed
down the toilet along with the extra money spent. B vitamins are naturally
present in numerous foods, and many other foods are enriched with th